<a href="https://colab.research.google.com/github/tetrar124/Safety_Science/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fastFM
!pip install -U mlxtend
!pip install -U pandas
!pip install rgf_python
#!pip install -U tensorflow-gpu
!pip install rgf-python
!pip install bayesian-optimization

     |████████████████████████████████| 491kB 45.9MB/s 
     |████████████████████████████████| 1.3MB 47.3MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
     |████████████████████████████████| 10.5MB 34.2MB/s 
     |████████████████████████████████| 235kB 20.1MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.25.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: python-dateutil 2.5.3
    Uninstalling python-dateutil-2.5.3:
      Successfully uninstalled python-dateutil-2.5.3
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


     |████████████████████████████████| 768kB 40.4MB/s 
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10031 sha256=939a7dd6a9d20293676c8bcc4000c3342ce3dc3a73c5047a82151490d41e2e3b
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/colab/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/colab


In [5]:
import numpy as np
import itertools
import pandas as pd

from lightgbm import LGBMRegressor
from fastFM import sgd,als

from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.cross_decomposition import PLSRegression

from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import GridSearchCV,  cross_validate, StratifiedKFold
from sklearn.pipeline import make_pipeline

from mlxtend.regressor import StackingRegressor
from mlxtend.feature_selection import ColumnSelector
from rgf.sklearn import RGFRegressor
import xgboost
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import load_boston

import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Input, Dense, LSTM,CuDNNLSTM
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras import applications
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import accuracy_score

from sklearn.base import BaseEstimator, TransformerMixin,RegressorMixin
from sklearn.pipeline import make_pipeline

import scipy as sp
from sklearn.preprocessing import normalize
from sklearn.datasets import load_boston
from sklearn.decomposition import PCA
os.chdir(r'/content/drive/My Drive/Data/Meram Chronic Data')
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
ejectCAS = ['10124-36-4', '108-88-3', '111991-09-4', '116-29-0', '120-12-7', '126833-17-8', '13171-21-6',
                    '1333-82-0', '137-30-4', '148-79-8', '1582-09-8', '1610-18-0', '2058-46-0', '2104-64-5',
                    '21725-46-2',
                    '2303-17-5', '25311-71-1', '25812-30-0', '298-00-0', '298-04-4', '314-40-9', '330-54-1',
                    '4170-30-3',
                    '4717-38-8', '50-00-0', '52645-53-1', '55406-53-6', '56-35-9', '56-38-2', '60207-90-1', '6051-87-2',
                    '62-53-3', '6317-18-6', '69-72-7', '7440-02-0', '7447-40-7', '7722-84-1', '7733-02-0', '7758-94-3',
                    '80844-07-1', '82657-04-3', '84852-15-3', '86-73-7', '9016-45-9', '99-35-4']

df =pd.read_csv('fishMorganMACCS.csv')
#df2=pd.read_csv('chronicMACCSKeys_tanimoto.csv')
#df2 = df2.drop(ejectCAS,axis=1).set_index('CAS').dropna(how='all', axis=1)
baseDf = df
extractDf =  df['CAS'].isin(ejectCAS)
df = df[~df['CAS'].isin(ejectCAS)]
#df = df.set_index('CAS')
#df = pd.concat([df,df2],axis=1, join_axes=[df.index]).reset_index()
y = df['logTox']
#dropList = ['CAS','toxValue','logTox','HDonor', 'HAcceptors', 'AromaticHeterocycles', 'AromaticCarbocycles', 'FractionCSP3']
dropList = ['CAS','toxValue','logTox']
X = df.drop(columns=dropList)
#Normalize
def normalize(X):
    changeList = []
    for i,name in enumerate(X.columns):
        if i <679:
            changeList.append((0,1))
        elif i > 692:
            changeList.append((0,1))
        else:
            #try:
            #name = float(name)
            #except:
            std =X[name].std()
            mean = X[name].mean()
            if std==0:
              X[name] = X[name]
            else:
              X[name] = X[name].apply(lambda x: ((x - mean) * 1 / std + 0))
            changeList.append((mean, std))
    return X, changeList
X2,_ = normalize(X)


In [0]:
        def calcRMSE(real, pred):
            RMSE = (np.sum((pred - real.tolist()) ** 2) / len(pred)) ** (1 / 2)
            return RMSE

        def calcCorr(real, pred):
            corr = np.corrcoef(real, pred.flatten())[0, 1]
            return corr
        from sklearn.metrics import make_scorer
        myScoreFunc = {'Accuracy': make_scorer(mean_squared_error),'RMSE': make_scorer(calcRMSE),
                       'Correlation coefficient': make_scorer(calcCorr)}

In [18]:
print(X.shape)
print(y.shape)

(1254, 5610)
(1254,)


In [0]:
Lambda(lambda x: x+1,x**10)([1,2,3,4])

In [0]:
    def dnnCalcACC(testmodel,X=X,name=None):
        def calcRMSE(real, pred):
            RMSE = (np.sum((pred - real.tolist()) ** 2) / len(pred)) ** (1 / 2)
            return RMSE
        def calcCorr(real, pred):
            corr = np.corrcoef(real, pred.flatten())[0, 1]
            return corr
        from sklearn.metrics import make_scorer
        myScoreFunc = {'RMSE': make_scorer(calcRMSE),
                       'Correlation coefficient': make_scorer(calcCorr)}
        cv = KFold(n_splits=10, shuffle=True, random_state=0)
        Scores = cross_validate(testmodel, X, y, cv=cv, scoring=myScoreFunc,return_train_score=True)
        RMSETmp = Scores['test_RMSE'].mean()
        CORRTmP = Scores['test_Correlation coefficient'].mean()
        trainRMSETmp = Scores['train_RMSE'].mean()
        trainCORRTmP = Scores['train_Correlation coefficient'].mean()
        print(name,'test', RMSETmp, CORRTmP)
        print(name,'train',trainRMSETmp, trainCORRTmP)

In [0]:
  class addDim(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Y = np.expand_dims(X, axis=2)
        return Y
  #inputs = Input(((X2.shape[1],1)))
  def LSTMmodel(X2=X2):
    inputs = Input((X2.shape[1],1))
    x = CuDNNLSTM(32,return_sequences=True)(inputs)
    x = CuDNNLSTM(64,return_sequences=True)(x)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x  = Dense(128, activation="relu")(x)
    predictions = Dense(1)(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer="adam",loss='mean_squared_error')
    return model
  #nn = make_pipeline(addDim(),model)

In [65]:
inputs

<tf.Tensor 'input_19:0' shape=(?, 5610, 1) dtype=float32>

In [0]:
        cv = KFold(n_splits=10, shuffle=True, random_state=0)
        X2 = np.array(X)
        y2 = np.array(y)
        print(X2.shape)
        folds = list(cv.split(X2))
        resultscores=[]
        for i,(train, test) in enumerate(cv.split(X2)):
          model = LSTMmodel()
          model.fit(np.expand_dims(X2[train], axis=2),y2[train])
          score = model.evaluate(np.expand_dims(X2[test],axis=2), y2[test])
          print("%s: %.2f%%" % (model.metrics_names[0], scores))
          resultscores.append(score)
          tf.keras.backend.clear_session()

(1254, 5610)
126/126 [==============================] - 1s 6ms/sample - loss: 2.7178
loss: 2.55%
126/126 [==============================] - 1s 6ms/sample - loss: 2.6643
loss: 2.55%
126/126 [==============================] - 1s 6ms/sample - loss: 2.5896
loss: 2.55%
126/126 [==============================] - 1s 6ms/sample - loss: 2.6583
loss: 2.55%
125/125 [==============================] - 1s 6ms/sample - loss: 2.6807
loss: 2.55%
125/125 [==============================] - 1s 6ms/sample - loss: 2.6305
loss: 2.55%
125/125 [==============================] - 1s 6ms/sample - loss: 2.7916
loss: 2.55%
125/125 [==============================] - 1s 6ms/sample - loss: 2.6033
loss: 2.55%
125/125 [==============================] - 1s 6ms/sample - loss: 6.0282
loss: 2.55%
 640/1129 [================>.............] - ETA: 6s - loss: 3.9523

In [86]:
resultscores

[2.468452075171092,
 1.435398610811385,
 1.1801072964592585,
 1.186723684507703,
 1.3473514127731323,
 1.3805641765594483,
 1.3650593090057372,
 1.2791805620193482,
 1.2453728828430175,
 1.0534589042663574]

In [0]:
    def calcACC(testmodel,X=X,name=None):
        def calcRMSE(real, pred):
            RMSE = (np.sum((pred - real.tolist()) ** 2) / len(pred)) ** (1 / 2)
            return RMSE

        def calcCorr(real, pred):
            corr = np.corrcoef(real, pred.flatten())[0, 1]
            return corr
        from sklearn.metrics import make_scorer
        myScoreFunc = {'RMSE': make_scorer(calcRMSE),
                       'Correlation coefficient': make_scorer(calcCorr)}
        cv = KFold(n_splits=10, shuffle=True, random_state=0)
        Scores = cross_validate(testmodel, X, y, cv=cv, scoring=myScoreFunc,return_train_score=True)
        RMSETmp = Scores['test_RMSE'].mean()
        CORRTmP = Scores['test_Correlation coefficient'].mean()
        trainRMSETmp = Scores['train_RMSE'].mean()
        trainCORRTmP = Scores['train_Correlation coefficient'].mean()
        print(name,'test', RMSETmp, CORRTmP)
        print(name,'train',trainRMSETmp, trainCORRTmP)

In [0]:
a= sp.sparse.csc_matrix(X2.values)
cv = KFold(n_splits=10, shuffle=True, random_state=0)
class sparseNorm(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Y = normalize(sp.sparse.csc_matrix(X.values))
        return Y
fm = sgd.FMRegression(n_iter=1000, init_stdev=0.1, l2_reg_w=0,l2_reg_V=0, rank=2, step_size=0.1)
fm = sgd.FMRegression(
            n_iter=4743,
            init_stdev=0.1,
            rank=100,
            l2_reg_w=0,
            l2_reg_V=0,
            step_size=0.1,
        )
pipe = make_pipeline(sparseNorm(),fm)


In [0]:
from sklearn import preprocessing
a= sp.sparse.csc_matrix(X2.values)
sparseX = preprocessing.normalize(a)
print(sparseX)

In [0]:

def optFMRegression(n_iter, init_stdev, rank, l2_reg_w, l2_reg_V,step_size):
    cv = KFold(n_splits=10, shuffle=True, random_state=0)
    score = cross_validate(
            sgd.FMRegression(
            n_iter=int(n_iter),
            init_stdev=init_stdev,
            rank=int(rank),
            l2_reg_w=l2_reg_w,
            l2_reg_V=l2_reg_V,
            step_size=step_size,
        ),
        sparseX, y,
        scoring='neg_mean_squared_error',
        cv=cv,n_jobs=-1)
    val = score['test_score'].mean()
    return val
score = optFMRegression(500,0,1,5,5,1)
score

-inf

In [0]:
def optFMRegression(n_iter, init_stdev, rank, l2_reg_w, l2_reg_V,step_size):
    cv = KFold(n_splits=10, shuffle=True, random_state=0)
    score = cross_validate(
            sgd.FMRegression(
            n_iter=int(n_iter),
            init_stdev=init_stdev,
            rank=int(rank),
            l2_reg_w=l2_reg_w,
            l2_reg_V=l2_reg_V,
            step_size=step_size,
        ),
        sparseX, y,
        scoring='neg_mean_squared_error',
        cv=cv,n_jobs=-1)
    val = score['test_score'].mean()
    return val

opt = BayesianOptimization(
        optFMRegression,
        {
            'n_iter': (500,5000),
            'init_stdev' : (0,20),
            'rank' : (1,100),
            'l2_reg_w' : (0.1,1),
            'l2_reg_V' : (0,0.1),
            'step_size' : (0.01,1)}
    )
opt.maximize(init_points=10,n_iter=100)
opt.max

In [0]:
X2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,20.3,21.3,22.3,23.3,24.3,25.3,26.3,27.3,28.3,29.3,30.3,31.3,32.3,33.3,34.3,35.3,36.3,37.3,38.3,39.3,40.3,41.3,42.3,43.3,44.3,45.3,46.3,47.3,48.3,49.3,50.3,51.3,52.3,53.3,54.3,55.3,56.3,57.3,58.3,59.3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
class sparseNorm(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        from sklearn import preprocessing
        Y = preprocessing.normalize(sp.sparse.csc_matrix(X.values))
        return Y
fm = sgd.FMRegression(
            n_iter=9943,
            init_stdev=0.1,
            rank=219,
            l2_reg_w=0,
            l2_reg_V=0.06454,
            step_size=0.1,
        )
# fm = sgd.FMRegression(
#             n_iter=4743,
#             init_stdev=0.1,
#             rank=100,
#             l2_reg_w=0,
#             l2_reg_V=0,
#             step_size=0.1,
#         )
pipe = make_pipeline(sparseNorm(), fm)
calcACC(pipe, X=X2)

None test 1.2541720510947238 0.6584044433349703
None train 1.0667708507173823 0.7909100866387624


In [0]:
def optFMRegression(n_iter,rank, l2_reg_w, l2_reg_V):
    cv = KFold(n_splits=10, shuffle=True, random_state=0)
    score = cross_validate(
            sgd.FMRegression(
            n_iter=n_iter,
            init_stdev=0.1,
            rank=int(rank),
            l2_reg_w=l2_reg_w,
            l2_reg_V=l2_reg_V,
            step_size=0.1,
        ),
        sparseX, y,
        scoring='neg_mean_squared_error',
        cv=cv,n_jobs=-1)
    val = score['test_score'].mean()
    return val

opt = BayesianOptimization(
        optFMRegression,
        {
            'n_iter': (1000,10000),
            'rank' : (1,1000),
            'l2_reg_w':(0,10),
            'l2_reg_V':(0,10),
            }
    )
opt.maximize(init_points=10,n_iter=100)
opt.max

|   iter    |  target   | l2_reg_V  | l2_reg_w  |  n_iter   |   rank    |
-------------------------------------------------------------------------
|  1        | -2.645    |  5.552    |  8.434    |  3.456e+0 |  51.34    |
|  2        | -2.728    |  7.015    |  8.225    |  9.659e+0 |  611.9    |
|  3        | -2.664    |  7.405    |  9.113    |  4.448e+0 |  299.7    |
|  4        | -2.676    |  5.971    |  6.375    |  5.296e+0 |  180.9    |
|  5        | -2.885    |  6.361    |  8.665    |  5.821e+0 |  905.3    |
|  6        | -2.821    |  2.463    |  5.221    |  5.173e+0 |  982.6    |
|  7        | -2.879    |  7.909    |  5.632    |  9.255e+0 |  486.2    |
|  8        | -2.703    |  1.415    |  6.475    |  2.269e+0 |  18.97    |
|  9        | -2.73     |  4.203    |  4.202    |  1.961e+0 |  151.7    |
|  10       | -2.748    |  6.82     |  3.247    |  6.016e+0 |  73.06    |
|  11       | -1.66     |  1.736e-0 |  2.189e-0 |  1e+04    |  1.0      |
|  12       | -2.756    |  10.0     | 

{'params': {'l2_reg_V': 0.06454808266719353,
  'l2_reg_w': 0.0,
  'n_iter': 9943.385736317281,
  'rank': 219.3830156897833},
 'target': -1.5833112158687874}

In [0]:
opt = BayesianOptimization(
        optFMRegression,
        {
            'n_iter':(500,5000),
            'init_stdev':(0,20),
            'rank':(1,100),
            'l2_reg_w':(0,10),
            'l2_reg_V':(0,10),
            'step_size':(0.01,1)
        }
    )

In [0]:
class addDim(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Y = np.expand_dims(X, axis=2)
        return Y
def NN_model(data= X):
  #data = np.expand_dims(data, axis=0)
  #data = np.reshape(X_train.values, (-1, 13, 1))
  inputs = Input(((data.shape[1],1)))
#   print(inputs)
  x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(inputs)
  #x = BatchNormalization()(x)
  #x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(x)

  #x = MaxPooling1D(1, padding='same')(x)
  #x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(x)
  #x = MaxPooling1D(2, padding='same')(x)
  #x = Conv1D(16,2,dilation_rate=2,padding="same", activation="relu")(x)
  #x = MaxPooling1D(2, padding='same')(x)
  #predictions = Conv1D(1,2,dilation_rate=2,padding="same", activation="relu")(x)

  x = Flatten()(x)
  #x = Dense(2048, activation="relu")(x)
#   x = Dropout(0.5)(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(512, activation="relu")(x)
#   x = Dense(256, activation="relu")(x)
#   x = Dense(64, activation="relu")(x)
#   x = Dense(16, activation="relu")(x)
  x = Dropout(0.3)(x)
  x  = Dense(2048, activation="relu")(x)
  predictions = Dense(1)(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer="adam",loss='mean_squared_error')
  model.summary()
  return model
#nn_model = NN_model(X_train)
nn = KerasRegressor(build_fn=NN_model, epochs=100, batch_size=100, verbose=0)
nn = make_pipeline(addDim(),nn)

In [0]:
class addDim(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Y = np.expand_dims(X, axis=2)
        return Y
def LSTM_model(data= X):
  #data = np.expand_dims(data, axis=0)
  #data = np.reshape(X_train.values, (-1, 13, 1))
  inputs = Input(((data.shape[1],1)))
#   print(inputs)
#   x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(inputs)
#   x = BatchNormalization()(x)
  #x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(x)
  x = CuDNNLSTM(32,return_sequences=True)(inputs)
  x = CuDNNLSTM(64,return_sequences=True)(x)
  #x = MaxPooling1D(1, padding='same')(x)
  #x = Conv1D(1,1,dilation_rate=2,padding="same", activation="relu")(x)
  #x = MaxPooling1D(2, padding='same')(x)
  #x = Conv1D(16,2,dilation_rate=2,padding="same", activation="relu")(x)
  #x = MaxPooling1D(2, padding='same')(x)
  #predictions = Conv1D(1,2,dilation_rate=2,padding="same", activation="relu")(x)

  x = Flatten()(x)
  #x = Dense(2048, activation="relu")(x)
#   x = Dropout(0.5)(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(512, activation="relu")(x)
#   x = Dense(256, activation="relu")(x)
#   x = Dense(64, activation="relu")(x)
#   x = Dense(16, activation="relu")(x)
  x = Dropout(0.3)(x)
  x  = Dense(128, activation="relu")(x)
  predictions = Dense(1)(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer="adam",loss='mean_squared_error')
  model.summary()
  return model
nn = KerasRegressor(build_fn=LSTM_model, epochs=100, batch_size=100, verbose=0)
lstm = make_pipeline(addDim(),nn)

In [0]:
calcACC(lstm,X=X2)

W0626 15:45:38.144906 139993621567360 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5610, 1)]         0         
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 5610, 32)          4480      
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 5610, 64)          25088     
_________________________________________________________________
flatten (Flatten)            (None, 359040)            0         
_________________________________________________________________
dropout (Dropout)            (None, 359040)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               45957248  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129   

In [0]:
def NN_model2(data= X_train):
  inputs = Input(shape=data.shape[1:])
  #inputs = Input(shape=(7,))
  x = Dense(2048, activation="relu")(inputs)
#   x = Dropout(0.5)(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(2048, activation="relu")(x)
#   x = Dense(512, activation="relu")(x)
#   x = Dense(256, activation="relu")(x)
#   x = Dense(64, activation="relu")(x)
#   x = Dense(16, activation="relu")(x)
  #x = Dropout(0.3)(x)
  x = Dense(2048, activation="relu")(x)
  predictions = Dense(1)(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer="adam",loss='mean_squared_error')
  model.summary()
  return model
nn_model = NN_model2(X_train)
nn2 = KerasRegressor(build_fn=NN_model2, epochs=100, batch_size=10, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 690)               0         
_________________________________________________________________
dense (Dense)                (None, 2048)              1415168   
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 5,613,569
Trainable params: 5,613,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.preprocessing import normalize 

class sparseNorm(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Y = normalize(sp.sparse.csc_matrix(X.values))
        return Y
lgbm = LGBMRegressor(boosting_type='gbdt', num_leaves= 60,learning_rate=0.06)
rgf = RGFRegressor(max_leaf=1000, algorithm="RGF",test_interval=100, loss="LS",verbose=False,l2=1.0)
rgf1 = RGFRegressor(max_leaf=1000, algorithm="RGF",test_interval=100, loss="LS",verbose=False,l2=1.0)
rgf2 = RGFRegressor(max_leaf=1000, algorithm="RGF",test_interval=100, loss="LS",verbose=False,l2=1.0)
rgf3 = RGFRegressor(max_leaf=1000, algorithm="RGF",test_interval=100, loss="LS",verbose=False,l2=1.0)
rgf4 = RGFRegressor(max_leaf=1000, algorithm="RGF",test_interval=100, loss="LS",verbose=False,l2=1.0)

pipe1 = make_pipeline(extMACCS(), rgf)
pipe2 = make_pipeline(extMorgan(), rgf1)
pipe3 = make_pipeline(extDescriptor(), rgf2)
pipe4 = make_pipeline(extPCA(), rgf3)
pipe7 =make_pipeline(extDescriptor(), rgf4)
pipe8 =make_pipeline(extDescriptor(), rgf4)
from fastFM import sgd,als

fm = sgd.FMRegression(n_iter=1000, init_stdev=0.1, l2_reg_w=0,l2_reg_V=0, rank=2, step_size=0.1)
fmpipe = make_pipeline(sparseNorm(),fm)

ave = extAverage()
xgb = xgboost.XGBRegressor()
nbrs = KNeighborsRegressor(2)
svr = SVR(gamma='auto',kernel='linear')
sgd = SGDRegressor(max_iter=1000)
pls = PLSRegression(n_components=3)
ext = ExtraTreesRegressor(n_estimators=30,max_features= 20,min_samples_split= 5,max_depth= 50, min_samples_leaf= 5)

pipe5 = make_pipeline(extMorgan(), nbrs)
pipe6 = make_pipeline(extMACCS(), rgf)
alldata = make_pipeline(extAll())

meta = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=400)

pipe1 = make_pipeline(extMACCS(), meta)
pipe2 = make_pipeline(extMorgan(), meta)
pipe3 = make_pipeline(extDescriptor(), meta)
pipe4 = make_pipeline(extPCA(), rgf3)

pipe7 =make_pipeline(extDescriptor(), rgf4)
pipe8 =make_pipeline(extDescriptor(), rgf4)

stack = StackingRegressor(regressors=[pipe1,pipe2,pipe3,rgf,xgb,lgbm,meta], meta_regressor=ave, verbose=1)

stack1 = StackingRegressor(regressors=[pipe1, pipe2, pipe3,nn,fmpipe,nn2], meta_regressor=rgf, verbose=1)
#stack2 = StackingRegressor(regressors=[stack1,nbrs, svr,pls,rgf], meta_regressor=lgbm, verbose=1)
stack2 = StackingRegressor(regressors=[stack1,pipe5,pipe7,pipe1], meta_regressor=rgf,verbose=1)
cv = KFold(n_splits=10, shuffle=True, random_state=0)


In [0]:
#0.70
pipeRgf = make_pipeline(extMorgan(), rgf)
pipeXgb = make_pipeline(extMorgan(), xgb)
pipeLgbm = make_pipeline(extMorgan(), lgbm)
pipeNbrs = make_pipeline(extMorgan(), nbrs)


#stack = StackingRegressor(regressors=[pipe1,pipe2,pipe3,pipeRgf,pipeXgb,pipeLgbm], meta_regressor=ave, verbose=1)
stack = StackingRegressor(regressors=[meta,rgf,xgb,lgbm], meta_regressor=ave, verbose=1)
scores = cross_validate(stack, X, y, cv=cv)
score = scores['test_score'].mean()**(1/2)
score2 = scores['train_score'].mean()**(1/2)

In [0]:
score

0.6992734576657249

In [0]:
#0.68
pipePCA = make_pipeline(extPCA())

stack1 = StackingRegressor(regressors=[pipe1, pipe2, pipe3], meta_regressor=meta, verbose=1)
#0.68
stack2 = StackingRegressor(regressors=[stack1,alldata,pipePCA,nbrs], meta_regressor=rgf,verbose=1)

stack2 = StackingRegressor(regressors=[stack1,alldata,pipePCA,nbrs], meta_regressor=ave,verbose=1)
scores2 = cross_validate(stack2, X, y, cv=cv)
score3 = scores2['test_score'].mean()**(1/2)
score4 = scores2['train_score'].mean()**(1/2)

In [0]:
stack1 = StackingRegressor(regressors=[pipe1, pipe2, pipe3], meta_regressor=meta, verbose=1)
stack2 = StackingRegressor(regressors=[stack1,alldata], meta_regressor=meta,verbose=1)
scores3 = cross_validate(stack2, X, y, cv=cv)
score5 = scores3['test_score'].mean()**(1/2)
score6 = scores3['train_score'].mean()**(1/2)

In [0]:
score3

nan

In [0]:
pipePCA = make_pipeline(extPCA())

stack3 = StackingRegressor(regressors=[pipe1, pipe2, pipe3], meta_regressor=meta, verbose=1)
stack4 = StackingRegressor(regressors=[stack1,alldata,pipePCA], meta_regressor=rgf,verbose=1)
scores10 = cross_validate(stack4, X, y, cv=cv)
score11 = scores10['test_score'].mean()**(1/2)
score12 = scores10['train_score'].mean()**(1/2)

In [0]:
,rgf,xgb,lgbm,meta

In [0]:
stack = StackingRegressor(regressors=[pipe1,pipe2,pipe3], meta_regressor=ave, verbose=1)
scores = cross_validate(stack, X, y, cv=cv)
score = scores['test_score'].mean()**(1/2)
score2 = scores['train_score'].mean()**(1/2)

In [0]:
score

0.6785203788102289

In [0]:
singleResult = {}
for name in [xgb,nbrs,svr,sgd,pls,ext,lgbm,rgf,meta,fmpipe]:
  scores = cross_validate(name, X, y, cv=cv)
  score = scores['test_score'].mean()**(1/2)
  score2 = scores['train_score'].mean()**(1/2)
  name2 = str(name)
  singleResult[name] = (score,score2)

In [0]:
scores

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.02854085, 0.01813841, 0.01846361, 0.01905918, 0.01943231,
        0.01915932, 0.01916122, 0.0187676 , 0.0185051 , 0.01845622]),
 'score_time': array([0.00297284, 0.00268865, 0.00284624, 0.00341225, 0.00285077,
        0.00278807, 0.00270033, 0.00278497, 0.00265265, 0.00293851]),
 'test_score': array([0.09979622, 0.1815943 , 0.17128182, 0.18970238, 0.31300673,
        0.25077943, 0.25094152, 0.29138371, 0.28893897, 0.06266754]),
 'train_score': array([0.18613029, 0.12032096, 0.21351814, 0.22544877, 0.30175647,
        0.24757158, 0.35747028, 0.36475411, 0.35926899, 0.23259555])}

In [0]:
singleResult

{ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=50,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False): (0.6231301383871978,
  0.7680167562613793),
 KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform'): (0.5561273537929962, 0.8776402432176893),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.06, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=60, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=

In [0]:
lstm.fit(X_train, y_train)
y_pred = lstm.predict(X_train)
y_val = lstm.predict(X_test)
print("Root Mean Squared Error train: %.4f" % calcRMSE(y_pred, y_train))
print("Root Mean Squared Error test: %.4f" % calcRMSE(y_val, y_test))
print('Correlation Coefficient train: %.4f' % calcCorr(y_pred, y_train))
print('Correlation Coefficient test: %.4f' % calcCorr(y_val, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 690, 1)            0         
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 690, 224)          203392    
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 690, 384)          936960    
_________________________________________________________________
flatten_3 (Flatten)          (None, 264960)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 264960)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               33915008  
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total para

In [0]:
print(y_val,y_val.shape)
y_test.shape

[0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125
 0.14148125 0.14148125 0.14148125 0.14148125 0.14148125 0.1414

(126,)

In [0]:
scores = cross_validate(nn, X, y, cv=cv)
scores['test_score'].mean()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_71 (InputLayer)        (None, 690, 1)            0         
_________________________________________________________________
conv1d_93 (Conv1D)           (None, 690, 1)            2         
_________________________________________________________________
flatten_68 (Flatten)         (None, 690)               0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 690)               0         
_________________________________________________________________
dense_139 (Dense)            (None, 2048)              1415168   
_________________________________________________________________
dense_140 (Dense)            (None, 1)                 2049      
Total params: 1,417,219
Trainable params: 1,417,219
Non-trainable params: 0
_________________________________________________________________


-1.55726209964071

In [0]:
scores

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([17.53997922, 17.51591349, 17.62641382, 18.63325047, 18.35314131,
        18.77135444, 19.03872037, 19.38783813, 20.17999578, 19.95681596]),
 'score_time': array([2.63622236, 2.81975174, 2.88187885, 2.97793388, 3.18758726,
        3.28418374, 3.39250493, 3.42442966, 3.53948808, 3.70629454]),
 'test_score': array([-1.87397503, -1.48971084, -1.21239327, -1.56403474, -2.68844223,
        -2.90050378, -2.04052038, -1.72630579, -1.75029235, -1.57858052]),
 'train_score': array([-0.05194089, -1.44330622, -0.09889857, -1.31023485, -2.67425018,
        -2.65073041, -1.82119017, -1.13014557, -0.08168194, -1.48566212])}

In [0]:
scores = cross_validate(stack2, X, y, cv=10)
scores['test_score'].mean()


Fitting 4 regressors...
Fitting regressor1: stackingregressor (1/4)
Fitting 6 regressors...
Fitting regressor1: pipeline (1/6)
Fitting regressor2: pipeline (2/6)
Fitting regressor3: pipeline (3/6)
Fitting regressor4: pipeline (4/6)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 690, 1)            0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 690, 1)            2         
_________________________________________________________________
flatten_21 (Flatten)         (None, 690)               0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 690)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 2048)              1415168   
__________________________________________

array([0.19763957, 0.28065453, 0.37411911, 0.39152718, 0.4430288 ,
       0.48997388, 0.60091479, 0.48158644, 0.43197961, 0.22167769])

In [0]:
scores.mean()

0.39131016006993274

In [0]:
stack_scores =cross_validate(stack1, X, y, cv=10)
stack_scores

Fitting 6 regressors...
Fitting regressor1: pipeline (1/6)
Fitting regressor2: pipeline (2/6)
Fitting regressor3: pipeline (3/6)
Fitting regressor4: pipeline (4/6)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_50 (InputLayer)        (None, 690, 1)            0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 690, 1)            2         
_________________________________________________________________
flatten_34 (Flatten)         (None, 690)               0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 690)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 2048)              1415168   
_________________________________________________________________
dense_114 (Dense)            (None, 1)      

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([133.56943321, 135.46863294, 135.77038169, 136.78971791,
        138.16432285, 139.50198388, 140.6641326 , 141.19012785,
        143.97067618, 144.0953393 ]),
 'score_time': array([0.35180521, 0.35023689, 0.35720897, 0.36406612, 0.35870647,
        0.37927723, 0.38316083, 0.36888576, 0.38707685, 0.36656427]),
 'test_score': array([0.30323613, 0.26691208, 0.33824475, 0.4089086 , 0.38580084,
        0.47605946, 0.60830864, 0.5006599 , 0.40077139, 0.16564585]),
 'train_score': array([0.983355  , 0.99007855, 0.98353844, 0.9835329 , 0.9840576 ,
        0.98359359, 0.9835808 , 0.9813044 , 0.98336132, 0.98668905])}

In [0]:
stack_scores['test_score'].mean()

0.3854547633234017

In [0]:
rgf_scores=cross_validate(rgf, X, y, cv=10)
print(rgf_scores)
rgf_scores.mean()

[0.34539658 0.37707443 0.40949323 0.40390886 0.42384531 0.45379597
 0.6097257  0.42279969 0.46854228 0.49224834]


0.4406830390639035